In [1]:
import pandas as pd
import spacy
import json
from tqdm import tqdm

2023-01-17 16:20:40.639929: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 16:20:41.206754: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2023-01-17 16:20:41.206798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2023-01-17 16:20:41.206803: W tensorflow/compiler/tf2tensorrt/utils/py_utils

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)


In [3]:
#import the pipeline class


from InformationExtraction import InformationExtractionPipeline


# example spacy extractor function
NER = spacy.load("en_core_web_lg")
def tag_extraction_from_spacy(sen, model = NER):
    
    annotated = model(sen)
    extracted_names = [word.text for word in annotated.ents 
                       if word.label_=='PERSON' or word.label_=='ORG'or word.label_=='GPE']
    
    
    return extracted_names


def tag_extraction_from_LM(sen, lang, model = nlp):
    
    ner_results = model(sen)
    extracted_names = []
    for idx in range(len(ner_results)):
        if ner_results[idx]['entity'][0] == 'B':
            start = ner_results[idx]['start']
            end = ner_results[idx]['end']
            j = idx+1
            while j < len(ner_results):
                if ner_results[j]['entity'][0] == 'B':
                    break
                elif ner_results[j]['entity'][0] == 'I':
                    end = ner_results[j]['end']
                j+=1
            idx = j
        
            extracted_names.append(sen[start:end].strip())
    
    
    return extracted_names


# example extractor function that uses training labels 
# sent_to_tag = dict(zip(data['sent'],data['labels']))
def tag_extraction_from_tags(sent, lang, sent_to_tag):

    tags = sent_to_tag[sent]
    
    
    if lang=='zh':
        sentsWithtags = [(s,t) for s,t in zip(sent,tags.split())]
    else:
        sentsWithtags = [(s,t) for s,t in zip(sent.split(),tags.split())]
        
        
    entity_list = []
    for i,item in enumerate(sentsWithtags):
        if 'B-' in item[1]:
            j = i
            entity = []
            while j<len(sentsWithtags):
                if sentsWithtags[j][1] =='O':
                    break
                entity.append(sentsWithtags[j][0])
                j+=1
            i = j
             
            if lang=='zh':
                entity_list.append("".join(entity))
            
            else:
                entity_list.append(" ".join(entity))
            
    

    return entity_list




In [4]:
def get_info(Set, lang):
    data = pd.read_csv(f'../Dataset/{Set}.csv')
    data = data[data['lang']==lang]
    
    if lang=='zh':
        data['sent'] = data['sent'].apply(lambda x: "".join(x.split()))
            
    with open(f'./Wiki-jsons/wiki-info-{lang}-{Set}.json','r') as file:
        wiki = json.load(file)
        wiki = list(wiki.keys())

    sent_to_tag = dict(zip(data['sent'],data['labels']))
    detectedLM = 0
    detectedTags = 0
    totalTags = 0
    totalLM = 0
    irrelevent = 0
    common  = 1
    for sent in tqdm(data['sent']):


            
        tags = tag_extraction_from_tags(sent, lang, sent_to_tag = sent_to_tag)
        
        if lang!='en':
            lm = tag_extraction_from_LM(sent,lang)
        else:
            lm = tag_extraction_from_spacy(sent)

        if lang!='zh':
            tags = ["_".join([token.capitalize() for token in names.strip().split()]) for names in tags]
            lm = ["_".join([token.capitalize() for token in names.strip().split()]) for names in lm]
        
        totalTags += len(tags)
        totalLM += len(lm)
        
        for item in tags:
            if item in lm and item in wiki:
                detectedLM+=1
            if item in wiki:
                detectedTags+=1
            if item in lm:
                common += 1 
                
        for item in lm:
            if item not in tags and item in wiki:
                irrelevent+=1
                
        
                
                
    return totalLM, detectedLM, irrelevent, totalTags, detectedTags, common

In [10]:
Sets = ['train', 'dev']
data = pd.read_csv(f'../Dataset/train.csv')
# langs = data.lang.unique()
langs = ['zh']
pcts = []
for Set in Sets:
    
    for lang in langs:
        

        totalLM, detectedLM, irrelevent, totalTags, detectedTags, common = get_info(Set,lang)
    
        info = [lang, Set, totalTags , detectedTags, common, totalLM, detectedLM, irrelevent]
        print(f"Lang: {lang}, Set: {Set}, Total-LM: {totalLM}, found-LM: {detectedLM}, Toatal-Tags: {totalTags}, found-tags: {detectedTags}")
        pcts.append(info)
            

100%|███████████████████████████████████████| 9758/9758 [06:29<00:00, 25.06it/s]


Lang: zh, Set: train, Total-LM: 13114, found-LM: 2755, Toatal-Tags: 15226, found-tags: 6553


100%|█████████████████████████████████████████| 505/505 [00:39<00:00, 12.69it/s]

Lang: zh, Set: dev, Total-LM: 662, found-LM: 141, Toatal-Tags: 772, found-tags: 297


In [11]:
pcts

[['zh', 'train', 15226, 6553, 4308, 13114, 2755, 4796],
 ['zh', 'dev', 772, 297, 247, 662, 141, 199]]

In [7]:
# df = pd.DataFrame(pcts, columns=['Lang','Set','totalTags' , 'detectedTags', 'common', 'totalLM', 'detectedLM', 'irrelevent'])
# df.to_csv('info-label.csv',index=False)